In [1]:
import numpy as np
import pandas as pd
import gensim
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import re

# nltk.download('stopwords')
# nltk.download('wordnet')
# !pip install -U gensim
# nltk.download('punkt')
stopwords = stopwords.words('english')

C:\Users\hp\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# Loading the datasets

guard_df = pd.read_csv('guardian.csv', sep=',')

columns = ['firstPublicationDate', 'webUrl', 'bodyText']
guard_df = guard_df[columns]
guard_df.drop_duplicates(subset ="bodyText", inplace = True)

print(guard_df.shape)
guard_df.dropna(inplace=True)

guard_df.head()

For NYT Original Number of Columns: 23, rows: 19927
After droping duplicate rows and unneccesary columns
 Number of Columns: 3        , rows: 19742


In [3]:
# Pre-processing steps

%timeit
# initalizing the werdnet lemmatizer
lm = WordNetLemmatizer()
# creating a new column to store rows after processing
guard_df['post_processed'] = np.nan

stopwords.extend(['advertisement', 'supported'])
def processing(content):

    content = content.split(' ')
#   removing stopwords
    content = [word.strip().lower() for word in content if word.lower() not in stopwords]
#   removing punctuations
    rx = re.compile('([&#.:?!-()])*')
    content = [rx.sub('', word) for word in content]
    
    content = [word for word in content if len(word)>1 and word.isalpha()]
#   lemmatizing    
    content = [lm.lemmatize(word) for word in content]

    return ' '.join(content)

# removing opinion articles
for i in range(len(guard_df)):
    if 'opinion' not in guard_df.iloc[i,1]:
        guard_df.iloc[i,-1] = processing(guard_df.iloc[i,2]) 


guard_df.dropna(inplace=True)
guard_df.head()

After Removing Opinion articles, rows are reduced to :  17228


,pub_date,web_url,content,post_processed
1,2020-03-01T16:58:17+0000,https://www.nytimes.com/2020/03/01/upshot/coro...,Advertisement Supported by A sick day? Remote ...,sick day remote work easy job day care center ...
2,2020-03-01T18:10:26+0000,https://www.nytimes.com/2020/03/01/health/coro...,Advertisement Supported by Two cases detected ...,two case detected week apart washington state ...
3,2020-03-01T04:04:45+0000,https://www.nytimes.com/2020/02/29/us/trump-co...,Advertisement Supported by President Trump fou...,president trump found veering message discussi...
4,2020-03-01T12:00:08+0000,https://www.nytimes.com/2020/03/01/business/th...,Advertisement Supported by with interest By Da...,interest david going tax week said it maybe ac...
5,2020-03-01T08:00:08+0000,https://www.nytimes.com/2020/03/01/world/europ...,Advertisement Supported by Inside the outbreak...,inside outbreak british film crew shooting ita...


In [4]:
import gensim.corpora as corpora
#decomposing sentences into tokens 
tokens = [sentence.split(' ') for sentence in df['post_processed'] ]

# Create Dictionary
dct = corpora.Dictionary(tokens)
print('Unique words before filtering/after pre-processing', len(dct))
# no_below= 30
dct.filter_extremes(no_below= 20, no_above=0.25 )
print('Unique words after filtering', len(dct))
# Create Corpus
corpus = [dct.doc2bow(token) for token in tokens]

lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dct, num_topics=7, 
                                           random_state=100, chunksize=128, passes=10 )

Unique words before filtering/after pre-processing 97388
Unique words after filtering 15357


In [6]:
from gensim.test.utils import datapath

# temp_file = datapath("lda.model")
lda_model.save('lda.model')
# Load a potentially pretrained model from disk.
# lda_model1 = gensim.models.LdaModel.load('lda.model')
dct.save('dictionary')